# Evaluation

In [1]:
%load_ext autoreload
%autoreload 2

In [123]:
import pandas as pd
import numpy as np
from src import preprocess as prep
import random
from src.Video import Video
from src.Segment import Segment

## Load training set

In [152]:
num_videos = 1

training_set_generator = prep.load_training_set(range(1, num_videos+1))

In [153]:
training_set = list(training_set_generator)

In [154]:
# Print statistics
print("TRAINING SET:")
print("Num. videos:   {:d}".format( len(training_set)) )
print("Num. segments: {:d}".format( np.sum([len(video.segments) for video in training_set])) )
print("Duration:      {:,.1f} s".format( np.sum([np.sum([segment.duration() for segment in video.segments]) for video in training_set])) )

TRAINING SET:
Num. videos:   1
Num. segments: 114
Duration:      268.4 s


## Select random test set

In [156]:
test_n_segments = 1000
test_set = []
labels = []

for i in range(test_n_segments):
    
    # Find random video
    video = random.choice(training_set)
    
    # Select random segment and add histogram to test set
    segment = random.choice(video.segments)
    test_set.append(segment.histograms)
    labels.append(segment)

In [157]:
# Print statistics
print("TEST SET:")
print("Num. histograms: {:d}".format( len(test_set) ))

TEST SET:
Num. histograms: 1000


<br><br>

## Run model on test set

In [260]:
results = []

for histogram in test_set:
    results.append(find(training_set, histogram))

In [261]:
def find(training_set : [Video], segment: np.ndarray) -> Segment:
    
    # TODO: actually find segment here
    return random.choice(random.choice(training_set).segments)

## Evaluate performance

In [262]:
movie_correct = 0
movie_wrong = 0

for segment, label in zip(results, labels):
    
    # Check if movie is correct
    if segment == label:
        movie_correct += 1
    else:
        movie_wrong += 1

total = movie_correct + movie_wrong
fraction = movie_correct / total if total > 0 else 0

print("Correct: {:d}".format(movie_correct))
print("Wrong:   {:d}".format(movie_wrong))
print("Total:   {:d}".format(total))
print("TPR:     {:.1f}%".format(movie_correct / total * 100))

Correct: 10
Wrong:   990
Total:   1000
TPR:     1.0%
